## Create Vector Endpoints and Index on embedding column (One-Time)

In [0]:
from databricks.vector_search.client import VectorSearchClient

vsc = VectorSearchClient(disable_notice=True)

# Create endpoint (only once, takes ~2–3 mins)
vsc.create_endpoint(name="databricks_vectorsearch_endpoint")


In [0]:
# (Optional) STEP 1B: Check if endpoint is online
vsc.get_endpoint(name="databricks_vectorsearch_endpoint")

In [0]:
%sql
    ALTER TABLE `docai-dbx`.gold.doc_embeddings SET TBLPROPERTIES (delta.enableChangeDataFeed = true)

In [0]:
# STEP 2: Register Delta Table as a Vector Index
catalog = "docai-dbx"
schema = "gold"
table = "doc_embeddings"
source_table = f"{catalog}.{schema}.{table}"

# Use a fully qualified index name
index_name = f"{catalog}.{schema}.doc_embeddings_index"

vsc.create_delta_sync_index(
    endpoint_name="databricks_vectorsearch_endpoint",
    index_name=index_name,
    source_table_name=source_table,
    primary_key="chunk_id",
    pipeline_type="TRIGGERED",
    embedding_source_column="embedding_str",
    embedding_model_endpoint_name="databricks-gte-large-en"
)


In [0]:
# (Optional) STEP 2B: Confirm the index is registered
vsc.get_index(
    endpoint_name="databricks_vectorsearch_endpoint",
    index_name=index_name)
